In [16]:
from transformers import BertTokenizer
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
df_dict = pd.read_csv("important_words.csv")

In [17]:
df_dict["tokens"] = [[] for _ in range(len(df_dict.index))]

In [18]:
for i in range(len(df_dict)):
    out = tokenizer(df_dict.iloc[i, 0], padding='do_not_pad', \
                    max_length = 512, truncation=True, return_tensors="pt")
    
    for tok in out["input_ids"][0][1:-1]:
        df_dict.iloc[i, 6].append(tok.item())
df_dict.to_pickle("important_tokens.tkl")

In [19]:
df_dict

,word,business,entertainment,sport,tech,politics,tokens
0,3-d,0,1,0,0,0,"[124, 118, 173]"
1,3d,0,1,0,0,0,"[124, 1181]"
2,access,0,0,0,1,0,[2469]
3,access to the internet,0,0,0,1,0,"[2469, 1106, 1103, 7210]"
4,account,1,1,0,1,0,[3300]
...,...,...,...,...,...,...,...
1805,zarzuela,0,1,0,0,0,"[195, 1813, 10337, 9945]"
1806,zero interest rates,1,0,0,0,0,"[6756, 2199, 5600]"
1807,zombie funds,1,0,0,0,0,"[16192, 4381]"
1808,zoo,0,1,0,0,0,[15003]


In [20]:
business_imp = df_dict[df_dict["business"] == 1]
business_imp_list = []
for i in range(len(business_imp)):
    business_imp_list.append(business_imp.iloc[i, 6])


In [21]:
entertainment_imp = df_dict[df_dict["entertainment"] == 1]
entertainment_imp_list = []
for i in range(len(entertainment_imp)):
    entertainment_imp_list.append(entertainment_imp.iloc[i, 6])

In [22]:
sport_imp = df_dict[df_dict["sport"] == 1]
sport_imp_list = []
for i in range(len(sport_imp)):
    sport_imp_list.append(sport_imp.iloc[i, 6])

In [23]:
tech_imp = df_dict[df_dict["tech"] == 1]
tech_imp_list = []
for i in range(len(tech_imp)):
    tech_imp_list.append(sport_imp.iloc[i, 6])

In [24]:
politics_imp = df_dict[df_dict["politics"] == 1]
politics_imp_list = []
for i in range(len(tech_imp)):
    politics_imp_list.append(politics_imp.iloc[i, 6])

In [25]:
datapath = '../bbc-text.csv'
df = pd.read_csv(datapath)

df["text_separate"] = [[] for _ in range(len(df.index))]
df["important_labels"] = [[] for _ in range(len(df.index))]
df["important_tokens"] = [[] for _ in range(len(df.index))]
df.to_pickle("bbc-text-with-important.pkl")

In [26]:
for i in range(len(df)):
    for str in df.iloc[i,1].split('. '):
        if '! ' in str:
            for idx, str_e in enumerate(str.split('! ')):
                if idx == len(str.split('! ')) - 1:
                    df.iloc[i,2].append(str_e + '!')
                else:
                    df.iloc[i,2].append(str_e + '.')
        elif '? ' in str:
            for idx, str_q in enumerate(str.split('? ')):
                if idx == len(str.split('? ')) - 1:
                    df.iloc[i,2].append(str_q + '?')
                else:
                    df.iloc[i,2].append(str_q + '.')
        else:
            df.iloc[i,2].append(str + '.')
    df.iloc[i,2][-1] = df.iloc[i,2][-1][:-1]
df.to_pickle("bbc-text-with-important.pkl")
df.head()

,category,text,text_separate,important_labels,important_tokens
0,tech,tv future in the hands of viewers with home th...,[tv future in the hands of viewers with home t...,[],[]
1,business,worldcom boss left books alone former worldc...,[worldcom boss left books alone former world...,[],[]
2,sport,tigers wary of farrell gamble leicester say ...,[tigers wary of farrell gamble leicester say...,[],[]
3,sport,yeading face newcastle in fa cup premiership s...,[yeading face newcastle in fa cup premiership ...,[],[]
4,entertainment,ocean s twelve raids box office ocean s twelve...,[ocean s twelve raids box office ocean s twelv...,[],[]


In [27]:
def mark_sublist_positions(list1, sublists):
    a = [0] * len(list1)
    imp_tokens = []
    for sublist in sublists:
        n = len(sublist)
        m = len(list1)
        for i in range(m - n + 1):
            if list1[i:i + n] == sublist:
                a[i:i + n] = [1] * n
                imp_tokens += sublist
    return a, imp_tokens

for i in range(len(df)):
    text = df.iloc[i, 2]
    if df.iloc[i, 0] == "business":
        imp_list = business_imp_list
    elif df.iloc[i, 0] == "entertainment":
        imp_list = entertainment_imp_list
    elif df.iloc[i, 0] == "sport":
        imp_list = sport_imp_list
    elif df.iloc[i, 0] == "tech":
        imp_list = tech_imp_list
    elif df.iloc[i, 0] == "politics":
        imp_list = politics_imp_list
    
    for j in range(len(text)):
        inputs_text = tokenizer(text[j], padding='do_not_pad', max_length = 512, \
                                truncation=True, return_tensors="pt")
        find_token, imp_tokens = mark_sublist_positions(inputs_text['input_ids'][0].tolist(),\
                                                        imp_list)
        df.iloc[i, 3].append([0] + find_token + [0])
        df.iloc[i, 4].append(imp_tokens)

In [28]:
df

,category,text,text_separate,important_labels,important_tokens
0,tech,tv future in the hands of viewers with home th...,[tv future in the hands of viewers with home t...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,...","[[1313], [], [1313], [], [1977], [], [], [1977..."
1,business,worldcom boss left books alone former worldc...,[worldcom boss left books alone former world...,"[[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,...","[[11438, 6054, 6054], [], [1419], [], [7230], ..."
2,sport,tigers wary of farrell gamble leicester say ...,[tigers wary of farrell gamble leicester say...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[3495, 2074, 4896], [1283], [], [], [1149], [..."
3,sport,yeading face newcastle in fa cup premiership s...,[yeading face newcastle in fa cup premiership ...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,...","[[4355, 4355, 2074, 1668], [3222, 3282, 1342, ..."
4,entertainment,ocean s twelve raids box office ocean s twelve...,[ocean s twelve raids box office ocean s twelv...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,...","[[3755], [7260], [], [], [], [3789], [], [], [..."
...,...,...,...,...,...
2220,business,cars pull down us retail figures us retail sal...,[cars pull down us retail figures us retail sa...,"[[0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0,...","[[3736, 6989, 6989], [], [6989], [6989], [9339..."
2221,politics,kilroy unveils immigration policy ex-chatshow ...,[kilroy unveils immigration policy ex-chatshow...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,...","[[2818, 2818], [5257, 2301], [1710], [], [], [..."
2222,entertainment,rem announce new glasgow concert us band rem h...,[rem announce new glasgow concert us band rem ...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,...","[[1467, 3838, 17799, 3870], [1467, 1505], [383..."
2223,politics,how political squabbles snowball it s become c...,[how political squabbles snowball it s become ...,"[[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[1741], [], [], [], [15081], [], [], [], [], ..."


In [29]:
df.to_csv("bbc-text-imp.csv")

In [30]:
df.to_pickle("bbc-text-importantLabeled.pkl")